In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
from sqlalchemy import create_engine
import geopandas as gpd
import seaborn as sns
import matplotlib as plt


In [ ]:
conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset={encoding}'.format(
    user='dreamteam', 
    password='dreamteam', 
    # Sunsetting enhanced server after analysis is complete, it's costing money!
    # host = 'dreamteam-enhanced.conr4khphv8c.us-east-2.rds.amazonaws.com', 
    host = 'dreamteam.conr4khphv8c.us-east-2.rds.amazonaws.com',
    port=3306, 
    db='nyhealth',
    encoding = 'utf8'
)
engine = create_engine(conn_string)

In [ ]:
# Dataset from NYC Open Data: https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas/cpf4-rkhq
!curl 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=GeoJSON' -o nyc-neighborhoods.geojson

#Correlation matrix for highly rated yelp versus highly rated health score

In [ ]:
# Load the shapefile
df_nyc = gpd.GeoDataFrame.from_file('nyc-neighborhoods.geojson')
# Limit the data to only Manhattan neighborhoods 
df_manhattan = df_nyc[df_nyc.boro_name =='Manhattan']
import matplotlib.pyplot as plt

In [ ]:
# combined - high rating, low health

query = '''
select *
from NYHEALTH n
join YELP y on y.phone = n.phone
where grade = 'c'
and longitude between '-73.9' and '-74.5'
and latitude between '40' and '41'
and rating > 4;

'''
df = pd.read_sql(query, con=engine)
df['longitude'] = df[['longitude']].apply(pd.to_numeric) #in db, column is object instead of float64
df['latitude'] = df[['latitude']].apply(pd.to_numeric)


manhattan_plot = df_manhattan.plot(linewidth=0.5, color='White', edgecolor='Black', figsize=(15, 10))

scatterplot = df.plot(kind='scatter',
                           x='longitude', y='latitude',
                           figsize=(15, 10), 
                          s=.5, 
                            alpha=0.1,
                           ax=manhattan_plot
                           )


sns.kdeplot(df.longitude, df.latitude,
            gridsize=200, n_levels=15, 
            shade=True, alpha=0.7,
            cmap=plt.cm.Reds,  shade_lowest=False,
            ax=scatterplot
            )

In [ ]:
# combined - high rating, high health

query = '''
select *
from NYHEALTH n
join YELP y on y.phone = n.phone
where grade = 'a'
and longitude between '-73.9' and '-74.5'
and latitude between '40' and '41'
and rating > 4
and CAMIS not in (select CAMIS from NYHEALTH where grade in ('c','b'));


'''
df = pd.read_sql(query, con=engine)
df['longitude'] = df[['longitude']].apply(pd.to_numeric) #in db, column is object instead of float64
df['latitude'] = df[['latitude']].apply(pd.to_numeric) #in db, column is object instead of float64


manhattan_plot = df_manhattan.plot(linewidth=0.5, color='White', edgecolor='Black', figsize=(15, 10))

scatterplot = df.plot(kind='scatter',
                           x='longitude', y='latitude',
                           figsize=(15, 10), 
                          s=.5, 
                            alpha=0.1,
                           ax=manhattan_plot
                           )


sns.kdeplot(df.longitude, df.latitude,
            gridsize=200, n_levels=15, 
            shade=True, alpha=0.7,
            cmap=plt.cm.Purples,  shade_lowest=False,
            ax=scatterplot
            )

#What is the average health score of restaurants in NYC area with a 4.5 stars or higher?

In [ ]:
query = '''
SELECT Y.zip_code AS zip_code, AVG(score) AS health_score
FROM YELP Y JOIN NYHEALTH N ON Y.phone=N.phone
    WHERE Y.zip_code IN ('10026', '10027', '10030', '10037', '10039', '10001', '10011', '10018', 
    '10019', '10020', '10036', '10029', '10035', '10010', '10016', '10017', '10022', '10012', '10013', 
    '10014', '10004', '10005', '10006', '10007', '10038', '10280', '10002', '10003', '10009', '10021', 
    '10028', '10044', '10065', '10075', '10128', '10023', '10024', '10025', '10031', '10032', '10033', '10034', '10040','10002')
    AND Y.rating >= 4.5
    GROUP BY zip_code
ORDER BY health_score DESC;
'''
df = pd.read_sql(query, con=engine)
df.set_index('zip_code')
df


In [ ]:
#average health score across all zip codes in Manhattan
df["health_score"].mean()

In [ ]:
#worst health score
df["health_score"].max()

In [ ]:
#best health score
df["health_score"].min()

In [ ]:
#horizontal bar chart showing health score by manhattan zip code
df.set_index('zip_code','health_score').plot(kind='barh', figsize=(15,10)).legend(bbox_to_anchor=(1,1))

#What are the top 5 violations amongst each price category?

In [ ]:
query3a = '''
SELECT `VIOLATION DESCRIPTION`, COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE price IS NOT NULL AND `VIOLATION DESCRIPTION` IS NOT NULL AND price = "$"
GROUP BY `VIOLATION DESCRIPTION`, price
ORDER BY num_inspections DESC
LIMIT 5;
'''
ratnorm1 = '''
SELECT COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE PRICE = "$"
GROUP BY price;
'''
db_join1 = pd.read_sql(query3a, con=engine)
db_join1.set_index('price')
ratnorm1 = pd.read_sql(ratnorm1, con=engine)
ratnorm1.set_index('price')
val = db_join1.num_inspections / ratnorm1.num_inspections[0]
db_join1["% inspect in price"] = val

In [ ]:
query3b = '''
SELECT `VIOLATION DESCRIPTION`, COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE price IS NOT NULL AND `VIOLATION DESCRIPTION` IS NOT NULL AND price = "$$"
GROUP BY `VIOLATION DESCRIPTION`, price
ORDER BY num_inspections DESC
LIMIT 5;
'''
ratnorm2 = '''
SELECT COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE PRICE = "$$"
GROUP BY price;
'''
db_join2 = pd.read_sql(query3b, con=engine)
db_join2.set_index('price')
ratnorm2 = pd.read_sql(ratnorm2, con=engine)
ratnorm2.set_index('price')
val2 = db_join2.num_inspections / ratnorm2.num_inspections[0]
db_join2["% inspect in price"] = val2
db2= db_join1.append(db_join2)

In [ ]:
query3c = '''
SELECT `VIOLATION DESCRIPTION`, COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE price IS NOT NULL AND `VIOLATION DESCRIPTION` IS NOT NULL AND price = "$$$"
GROUP BY `VIOLATION DESCRIPTION`, price
ORDER BY num_inspections DESC
LIMIT 5;
'''
ratnorm3 = '''
SELECT COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE PRICE = "$$$"
GROUP BY price;
'''
db_join3 = pd.read_sql(query3c, con=engine)
db_join3.set_index('price')
ratnorm3 = pd.read_sql(ratnorm3, con=engine)
ratnorm3.set_index('price')
val3 = db_join3.num_inspections / ratnorm3.num_inspections[0]
db_join3["% inspect in price"] = val3
db3 = db2.append(db_join3)

In [ ]:
query3d = '''
SELECT `VIOLATION DESCRIPTION`, COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE price IS NOT NULL AND `VIOLATION DESCRIPTION` IS NOT NULL AND price = "$$$$"
GROUP BY `VIOLATION DESCRIPTION`, price
ORDER BY num_inspections DESC
LIMIT 5;
'''
ratnorm4 = '''
SELECT COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE PRICE = "$$$$"
GROUP BY price;
'''
db_join4 = pd.read_sql(query3d, con=engine)
db_join4['price'] = "4$"
db_join4.set_index('price')
ratnorm4 = pd.read_sql(ratnorm4, con=engine)
ratnorm4.set_index('price')
val4 = db_join4.num_inspections / ratnorm4.num_inspections[0]
db_join4["% inspect in price"] = val4
dbfinal = db3.append(db_join4)

In [ ]:
dbfinal.set_index('price')
dbfinal['% inspect in price']= round(dbfinal['% inspect in price']*100,2)

In [ ]:
dfp = pd.pivot_table(data = dbfinal, index='VIOLATION DESCRIPTION', columns='price', values='% inspect in price')

In [ ]:
dfp

#Does more $$ = < rats?

In [ ]:
ratcount ='''
SELECT COUNT(*) AS rats, price, `VIOLATION DESCRIPTION`
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
WHERE `VIOLATION CODE`="04k" 
GROUP BY price;
'''
db_rats = pd.read_sql(ratcount, con=engine)

In [ ]:
#db_rats=db_rats.set_index('price')
db_rats.drop(db_rats.index[0], inplace=True)
db_rats

In [ ]:
db_rats=db_rats.drop(columns="VIOLATION DESCRIPTION")

In [ ]:
ratnorm = '''
SELECT COUNT(*) AS num_inspections, price
from YELP Y JOIN NYHEALTH N ON Y.phone = N.phone
GROUP BY price;
'''
norm = pd.read_sql(ratnorm, con=engine)
norm.drop(norm.index[0], inplace=True)
#norm=norm.set_index('price')

In [ ]:
v = (db_rats.rats / norm.num_inspections) * 100
db_rats["%_inspect"] = v

In [ ]:
db_rats

In [ ]:
dbplot = db_rats
dbplot['labels'] = ['$','2$','3$','4$']
dbplot.set_index('labels')
dplot1=dbplot.drop(columns=['price','%_inspect'])
dplot2=dbplot.drop(columns=['price','rats'])
dplot1.set_index('labels')

In [ ]:
dplot1_go=dplot1.plot(kind="barh", color="#d32323ff", title="", legend=False, figsize=(15,4))
dplot1_go.set_xlabel("# violations", fontsize=20)
dplot1_go.set_ylabel("$ category", fontsize=20)

In [ ]:
dplot2_go=dplot2.plot(kind="barh", color="#ffd58d", title="", legend=False,figsize=(15,4))
dplot2_go.set_xlabel("% violations",fontsize=20)
dplot2_go.set_ylabel("$ category",fontsize=20)

In [ ]:
query4 = '''
SELECT COUNT(*) AS number_of_inspections, review_count AS number_of_reviews
FROM YELP Y JOIN NYHEALTH N ON Y.phone=N.phone
GROUP BY review_count;
'''

In [ ]:
reviewsinspections = pd.read_sql(query4, con=engine)

In [ ]:
reviewsinspections.plot.area(stacked=False, figsize = (25,15)).legend(bbox_to_anchor=(.5, .5))

In [ ]:
clean_mask = (reviewsinspections.number_of_reviews < 3000) & (reviewsinspections.number_of_inspections < 3000) & (reviewsinspections.number_of_reviews > 0) & (reviewsinspections.number_of_inspections > 0)
cleandf = reviewsinspections[clean_mask]

In [ ]:
cleandf.plot.area(stacked=False, figsize = (25,15))